In [1]:
%matplotlib inline

In [2]:
import scipy as sp
import scipy.interpolate
import numpy as np
import glob #to read the files
from sklearn.decomposition import PCA
from ipywidgets import *
from tabulate import tabulate #to export in table format
from scipy import interpolate
import scipy.interpolate as interp
from scipy.integrate import quad
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import scipy.integrate as integrate
from astropy import constants as const
import sncosmo

In [3]:
csp = sncosmo.get_magsystem('csp')
Norm_V = (1/(csp.zpbandflux("cspv9844")))
Norm_r = (1/(csp.zpbandflux("cspr")))

In [4]:
filter_y_V = []
filter_z_V = []

In [5]:
file = open("V_tel_ccd_atm_ext_1.2.txt")
lines = file.readlines()
file.close()

In [6]:
for line in lines:

    if line.startswith('#'): continue
    co=line.rstrip().replace('INDEF','Nan').split()

    filter_y_V.append(co[0])
    filter_z_V.append(co[1])

filter_y_V = np.array(filter_y_V, dtype=float)
filter_z_V = np.array(filter_z_V, dtype=float)

In [7]:
f_filter_V = interp1d(filter_y_V, filter_z_V)

In [8]:
filter_y_r = []
filter_z_r = []

In [9]:
file = open("r_tel_ccd_atm_ext_1.2_new.txt")
lines = file.readlines()
file.close()

In [10]:
for line in lines:

    if line.startswith('#'): continue
    co=line.rstrip().replace('INDEF','Nan').split()

    filter_y_r.append(co[0])
    filter_z_r.append(co[1])

filter_y_r = np.array(filter_y_r, dtype=float)
filter_z_r = np.array(filter_z_r, dtype=float)

In [11]:
f_filter_r = interp1d(filter_y_r, filter_z_r)

In [12]:
x = np.linspace(-10,50,61)
y = np.linspace(3000,9000,601)

In [13]:
file = open("max_values.txt")
lines = file.readlines()
file.close()

snname = []
maxvalueV = []
maxvaluer = []

for line in lines:

    if line.startswith('#'): continue
    co=line.rstrip().replace('INDEF','Nan').split()

    snname.append(co[0])
    maxvalueV.append(co[1])
    maxvaluer.append(co[2])


maxvalueV = np.array(maxvalueV, dtype=float)
maxvaluer = np.array(maxvaluer, dtype=float)

In [14]:
#for i in range(0,len(snname)):
    
#    snname[i] = snname[i][:-5]

In [15]:
sn_names=glob.glob("*.dat")
print(len(sn_names))

113


In [16]:
for i in range(0,len(sn_names)):

    sn_names[i] = sn_names[i][:-4]

In [17]:
factorsV = []
namesV = []

In [18]:
factorsr = []
namesr = []

In [19]:
#hist = []

In [20]:
#sn_names = ["SN2012fr"]

In [21]:
for k in range(0,len(sn_names)):

    x_lc = []
    z_lc = []

    print(sn_names[k])

    N = sn_names[k]+".dat"


    file = open(N)
    lines = file.readlines()
    file.close()

    xtemp = []

    for line in lines:

        if line.startswith('#'): continue
        co=line.rstrip().replace('INDEF','Nan').split()

        xtemp.append(co[0])

    xtemp = np.array(xtemp, dtype=float)


    z = np.reshape(xtemp, (601, 61))

    zz = []
    xx = []
    yy = []

    for i in range(0,len(y)):

        for j in range(0,len(x)):

            zz.append(z[i][j])
            xx.append(x[j])
            yy.append(y[i])
            
            
            

    wavetest = []
    
    for i in range(0,len(zz)):
    
        if zz[i] != 0:

            wavetest.append(yy[i])
    
    minvar = False
    maxvar = False
    
    #teste da banda V
    for i in range(0,len(wavetest)):

        if 4600 -50 < wavetest[i] < 4600 + 50:
            minvar = True
            #print("possui minimo")

        if 6580 - 50 < wavetest[i] < 6580 + 50:
            maxvar = True
            #print("possui maximo")
    #print(minvar, maxvar)
    
    
    if maxvar == True and minvar == True:
        
        for j in range(0,len(x)):

            y_temp = []
            z_temp = []

            for i in range(0,len(xx)):

                if xx[i] == x[j]:

                    y_temp.append(yy[i])
                    z_temp.append(zz[i])

            f = interp1d(y_temp, z_temp)

            def S(x): return f(x)*f_filter_V(x)
            
            integral_value = integrate.quad(S, min(filter_y_V), max(filter_y_V))[0]
            
            z_lc.append(integral_value*Norm_V)
            
            x_lc.append(x[j]) 

        maxvaluecalc = max(z_lc)

        #print(maxvaluecalc)
        if sn_names[k] in snname:
            
            idx = snname.index(sn_names[k])
            
            if maxvalueV[idx] != -9999:

                #print("ok")
                factorsV.append(maxvalueV[idx]/maxvaluecalc)
                namesV.append(sn_names[k])
            
    # se nao ta no range da banda v entao tenta r
    if maxvar == False or minvar == False:
        
        print("nao ta na banda v")
        minvar = False
        maxvar = False

        #teste da banda V
        for i in range(0,len(wavetest)):

            if 5340 -50 < wavetest[i] < 5340 + 50:
                minvar = True
                #print("possui minimo")

            if 7190 - 50 < wavetest[i] < 7190 + 50:
                maxvar = True
                #print("possui maximo")
        #print(minvar, maxvar)
        
        if maxvar == True and minvar == True:
                print("teste para r sucesso")
                for j in range(0,len(x)):

                    y_temp = []
                    z_temp = []

                    for i in range(0,len(xx)):

                        if xx[i] == x[j]:

                            y_temp.append(yy[i])
                            z_temp.append(zz[i])

                    f = interp1d(y_temp, z_temp)

                    def S(x): return f(x)*f_filter_r(x)

                    integral_value = integrate.quad(S, min(filter_y_r), max(filter_y_r))[0]

                    z_lc.append(integral_value*Norm_r)

                    x_lc.append(x[j])

                maxvaluecalc = max(z_lc)

                if sn_names[k] in snname:

                    idx = snname.index(sn_names[k])

                    if maxvaluer[idx] != -9999:

                        #print("ok")
                        factorsr.append(maxvaluer[idx]/maxvaluecalc)
                        namesr.append(sn_names[k])

SN2000cx
SN2001ay
SN2001eh
SN1997bq
SN2003fa
SN2009ig
nao ta na banda v
SN2008bc
SN2005eu
SN2009D
SN2007ax
SN2006D
SN1999dq
SN1998bp
iPTF14bdn
ASASSN-14lp
SN2002fk
SN2003W
SN2008ia
SN1997br
SN2004gs
SN2009ad
SN2007F
SN2013dy
SN1999ac
SN2008ar
SN2002cs
SN2002cr
SN1997bp
2017hbi


<ipython-input-21-2decf5dabf5c>:86: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  integral_value = integrate.quad(S, min(filter_y_V), max(filter_y_V))[0]


SN2007bc
SN2005ke
SN2005hk
SN1996X
SN2005ki
SN2005cf
SN2006bt
SN2005el
SN1998dh
SN2007S
SN2004at
SN2000cn
SN2006S
SN1997E
SN2007bd
SN1972E
SN2002er
SN2002bz
SN2001en
SN2000dk
SN1994ae
SN2008C
SN2005am
SN2002bo
2011ay
SN2007on
SN2007cq
SN2005hc
SN1992A
LSQ12fxd
LSQ12gdj
SN2002de
SN2016coj
SN2001bf
2013e
SN2001bg
SN2015bp
SN1994D
SN2007qe
SN2002ha
iPTF13dge
SN2005na
SN2002dj
SN1999by
SN1999aa
SN1998aq
SN2006ax
SN2001gc
SN2002dp
SN2004ef
2012cg
SN2017fgc
SN2012hr
SN2009Y
SN1995D
SN2004as
SN2002fb
SN2008ec
SN2010ae
SN2008Z
SN2012fr
SN2007af
SN2006cp
SN2007kk
SN2005M
SN2007hj
SN2003cq
LSQ12gpw
SN2005kc
SN2001V
SN2006gr
SN2001cp
SN2007if
SN2003kf
SN1999ee
SN2013bh
SN2003du
SN2012ht
nao ta na banda v
SN2017erp
SN2001ep
SN2001fe
SN2008ha
nao ta na banda v
teste para r sucesso
SN2006kf
SN2008bf


In [24]:
namesr

['SN2008ha']

In [22]:
save_path = '/home/joao/Documentos/PCA'

In [25]:

table = []
for l in range(0,len(namesV)):
    table.append((namesV[l], factorsV[l]))

name_of_file = "factors"

completeName = os.path.join(save_path, name_of_file+".txt")

f = open(completeName, 'w')
f.write(tabulate(table, tablefmt="plain"))
f.close()


In [ ]:
factors